# IWV

In [8]:
from metacatalog import api
import pandas as pd
from collections import defaultdict

In [2]:
session = api.connect_database()

## Metadata

In [3]:
# Author
author = api.find_person(session, last_name='Yuan', return_iterator=True).first()
if author is None:
    author = api.add_person(
        session, 
        first_name='Peng',
        last_name='Yuan', 
        affiliation='Geodetic Institute (GIK), KIT',
        organisation_name='Karlsruhe Institute for Technology (KIT)',
        organisation_abbrev='KIT'
    )

print(author)

Peng Yuan <ID=6>


In [4]:
# variable
iwv = api.find_variable(session, symbol='iwv', return_iterator=True).first()
if iwv is None:
    unit = api.add_unit(session, name='Kilogram per squaremeter', symbol='kg*m^-2', si='kg*m^-2')
    iwv = api.add_variable(
        session,
        name='Integrated Water Vapor',
        symbol='iwv',
        column_names=['ivw'],
        unit=unit.id
    )
print(iwv)

Integrated Water Vapor [kg*m^-2] <ID=10005>


In [5]:
# Abstract
abstract = """
We obtained the one hourly time series of Integrated Water Vapor (IWV) for 66 stations of the GNSS Upper Rhine Graben network (GURN) network.
The time period is from January 1, 2015 to June 30, 2019.
The IWV were calculated with tropospheric Zenith Total Delay (ZTD) from Global Positioning System (GPS) at the stations.
Station pressure (Ps) and weighte mean temperature (Tm) were also computed from ERA5 reanalysis for the retieval of the GPS IWV.
"""

In [6]:
# lincenses
for lic in api.find_license(session):
    print(lic)
    
license = api.find_license(session, id=5)[0]
print('\n---------------\n', license)

Open Data Commons Open Database License <ID=4>
Open Data Commons Attribution License v1.0 <ID=5>
Creative Commons Attribution 4.0 International <ID=6>
Creative Commons Attribution-ShareAlike 4.0 International <ID=7>
Creative Commons Attribution-NonCommerical 4.0 International <ID=8>
Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International <ID=9>
Data licence Germany – attribution – version 2.0 <ID=10>

---------------
 Open Data Commons Attribution License v1.0 <ID=5>


In [19]:
# load the datafile
with open('./gik150010.txt', 'r') as f:
    raw = f.read()

# extract the stations
stations = defaultdict(lambda: dict())

current_block = None
for line in raw.splitlines():
    # Check for start blocks
    if '+SITE/ID' in line:
        current_block = 'site'
        continue
    if '+SITE/COORDINATES' in line:
        current_block = 'coords'
        continue
    if '+SITE/ECCENTRICITY' in line:
        current_block = 'eccen'
        continue

    # check for end blocks
    if '-SITE/ID' in line or '-SITE/COORDINATES' in line or '-SITE/ECCENTRICITY' in line:
        current_block = None
        continue

    # we do not process headers for now. They are hardcoded
    if line.startswith('*'):
        continue

    # process the blocks
    if current_block == 'site':
        chunks = line.split()
        stations[chunks[0]] = dict(location=f"SRID=4327;POINT({chunks[3]} {chunks[4]})", height=float(chunks[6]))
    
    if current_block == 'coords':
        c = line.split()
        stations[c[0]].update(dict(x=float(c[6]), y=float(c[7]), z=float(c[8]), system=c[9]))
    
    if current_block == 'eccen':
        c = line.split()
        HIER WEITER

stations

defaultdict(<function __main__.<lambda>()>,
            {'0384': {'location': 'SRID=4327;POINT(9.170927 48.779479)',
              'height': 292.628,
              'x': 4157306.964,
              'y': 671172.117,
              'z': 4774690.771,
              'system': 'IGS08'},
             '0386': {'location': 'SRID=4327;POINT(9.218325 49.138494)',
              'height': 186.204,
              'x': 4126955.605,
              'y': 669775.587,
              'z': 4800826.907,
              'system': 'IGS08'},
             '0388': {'location': 'SRID=4327;POINT(8.112590 48.830100)',
              'height': 137.199,
              'x': 4164699.479,
              'y': 593658.245,
              'z': 4778281.444,
              'system': 'IGS08'},
             '0389': {'location': 'SRID=4327;POINT(9.717587 49.106440)',
              'height': 384.473,
              'x': 4123747.466,
              'y': 706187.932,
              'z': 4798643.717,
              'system': 'IGS08'},
             '03

In [30]:
# build the Entry

entry = api.find_entry(session, title='GPS IWV product of the GURN network', author=author, return_iterator=True).first()

if entry is None:
    entry = api.add_entry(
        session,
        title='GPS IWV product of the GURN network',
        author=author.id,
        location='SRID=4326;POINT (7.5 48.7)',
        geom='SRID=4326;POLYGON ((5.2 46.9, 5.2 50.8, 9.8 50.8, 9.8 46.9, 5.2 46.9))',
        variable=iwv.id,
        abstract=abstract,
        license=license,
        embargo=False
    )

print(entry)

<ID=91 GPS IWV product of t [Integrated Water Vapor] >


In [31]:
# add data